In [29]:

import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.feature_extraction.text import CountVectorizer
from scipy import stats
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

df=pd.read_csv("firstTry.csv")
df


,predictor,rating_chooses_appropriate_action,rating_commits_to_action,rating_gathers_information,rating_identifies_issues_opportunities,rating_interprets_information,rating_involves_others,rating_decision_making_final_score
0,"Hi Tracy, This sounds very ionteresting, prio...",2,2,2,2,2,3,2
1,"Hi Tracy, I am happy to see we have a self mo...",4,3,4,3,4,2,6
2,J.J please can you work with her for two mor...,2,2,2,2,2,2,2
3,"Hi Paxton Let me talk to Debby first, before...",3,3,3,2,2,2,3
4,JJ HR along with myself and the Final Assem...,3,3,2,3,2,2,3
...,...,...,...,...,...,...,...,...
1376,"Dear Tracey,I wanted to take the time to than...",2,2,2,3,3,2,2
1377,Hi Tracy - thanks for your note and for shari...,4,4,3,4,4,4,6
1378,"Dear Tracey,Thanks for sharing the data.We do...",3,3,2,3,2,2,2
1379,"Hi Tracy, Thanks for sharing customer satisfa...",4,3,3,4,3,4,6


In [31]:
# Criterion 1 model

#Note: Once you find a satisfactory cross-validated score, you don't need to split the data, just run the model on everything
#and predict the validate.csv on the second chunk
y=df['rating_chooses_appropriate_action']
X_train, X_test, y_train, y_test= train_test_split(
                                                df['predictor'], y,
                                                test_size=0.33,
                                                random_state=53)

#count_vectorizer = CountVectorizer(stop_words='english')
#count_train = count_vectorizer.fit_transform(X_train.values)
#count_test = count_vectorizer.transform(X_test.values)

#Converting words to numbers
#Note: try other methods if you have time, like sentiment analysis, bag of words, or quanteda. 
#Note: You might need to change max_features if it gives you an error in the next chunk, when predicting the validate dataset
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7, max_features=13000)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

#Hyperparameter tuning
#Note: use less parameters or make the ranges shorter if it's taking too long
param_grid = {'max_depth':range(1, 7),
                'learning_rate':np.arange(0.005, 0.3, 0.07),
                 'n_estimators':np.arange(100, 2000, 200),
                 'gamma':np.arange(0, 0.06, 0.01),
                 'subsample':np.arange(0.5, 1, 0.1),
                 'colsample_bytree':np.arange(0.5, 1, 0.1)}

#Fitting model
#Note: Hillclimbing methods could be used instead of gridsearch. This might be quicker. 
#Note: randomizedSearchCV() could be used instead of GridSearchCV() to make the algorithm quicker.
#Note: try other models, like neural network classifier. We prefer a classification algorithm instead of regression. 
grid = GridSearchCV(xgb.XGBRegressor(random_state=0), 
                    param_grid=param_grid, cv=StratifiedShuffleSplit(100, random_state=1), verbose=3, scoring='neg_mean_squared_error')
    
grid.fit(tfidf_train, y_train)
pred1 = grid.predict(tfidf_test)

print("Best parameters:", grid.best_params_)
#Note: If you're using a classifier, you're going to have to change the metric to be the AUC or accuracy instead of the RMSE or r2
print("Lowest RMSE: ", (-grid.best_score_)**(1/2.0))
r2_score(y_test, pred1)
#Use this for classification
#metrics.accuracy_score(y_test, pred)

Fitting 10 folds for each of 225000 candidates, totalling 2250000 fits
[CV 1/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=100, subsample=0.5;, score=-2.082 total time=   0.6s
[CV 2/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=100, subsample=0.5;, score=-2.454 total time=   0.6s
[CV 3/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=100, subsample=0.5;, score=-2.104 total time=   0.6s
[CV 4/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=100, subsample=0.5;, score=-2.279 total time=   0.6s
[CV 5/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=100, subsample=0.5;, score=-2.360 total time=   0.6s
[CV 6/10] END 

[CV 5/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=100, subsample=0.8999999999999999;, score=-2.351 total time=   0.7s
[CV 6/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=100, subsample=0.8999999999999999;, score=-2.370 total time=   0.7s
[CV 7/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=100, subsample=0.8999999999999999;, score=-2.056 total time=   0.7s
[CV 8/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=100, subsample=0.8999999999999999;, score=-1.990 total time=   0.7s
[CV 9/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=100, subsample=0.8999999999999999;, score=-2.167 total time=   0.7s
[CV 10/10]

[CV 9/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=300, subsample=0.7999999999999999;, score=-0.579 total time=   2.1s
[CV 10/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=300, subsample=0.7999999999999999;, score=-0.828 total time=   2.1s
[CV 1/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=300, subsample=0.8999999999999999;, score=-0.528 total time=   2.3s
[CV 2/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=300, subsample=0.8999999999999999;, score=-0.662 total time=   2.2s
[CV 3/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=300, subsample=0.8999999999999999;, score=-0.499 total time=   2.2s
[CV 4/10]

[CV 3/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=500, subsample=0.7999999999999999;, score=-0.298 total time=   3.6s
[CV 4/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=500, subsample=0.7999999999999999;, score=-0.335 total time=   3.6s
[CV 5/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=500, subsample=0.7999999999999999;, score=-0.384 total time=   3.6s
[CV 6/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=500, subsample=0.7999999999999999;, score=-0.419 total time=   3.6s
[CV 7/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=500, subsample=0.7999999999999999;, score=-0.323 total time=   3.6s
[CV 8/10] 

[CV 7/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=700, subsample=0.7;, score=-0.302 total time=   5.0s
[CV 8/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=700, subsample=0.7;, score=-0.361 total time=   5.0s
[CV 9/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=700, subsample=0.7;, score=-0.348 total time=   5.1s
[CV 10/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=700, subsample=0.7;, score=-0.410 total time=   5.3s
[CV 1/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=1, min_child_weight=4.818676232053935, n_estimators=700, subsample=0.7999999999999999;, score=-0.320 total time=   5.1s
[CV 2/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, m

...

[CV 9/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=2, min_child_weight=4.818676232053935, n_estimators=300, subsample=0.7;, score=-0.573 total time=   2.9s
[CV 10/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=2, min_child_weight=4.818676232053935, n_estimators=300, subsample=0.7;, score=-0.819 total time=   3.0s
[CV 1/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=2, min_child_weight=4.818676232053935, n_estimators=300, subsample=0.7999999999999999;, score=-0.519 total time=   3.1s
[CV 2/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=2, min_child_weight=4.818676232053935, n_estimators=300, subsample=0.7999999999999999;, score=-0.652 total time=   3.0s
[CV 3/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=2, min_child_weight=4.818676232053935, n_estimators=300, subsample=0.7999999999999999;, score=-0.501 total time=   3.0s
[CV 4/10] END colsample_bytree=0.5, gam

[CV 3/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=2, min_child_weight=4.818676232053935, n_estimators=500, subsample=0.7;, score=-0.295 total time=   5.2s
[CV 4/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=2, min_child_weight=4.818676232053935, n_estimators=500, subsample=0.7;, score=-0.332 total time=   5.4s
[CV 5/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=2, min_child_weight=4.818676232053935, n_estimators=500, subsample=0.7;, score=-0.367 total time=   5.3s
[CV 6/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=2, min_child_weight=4.818676232053935, n_estimators=500, subsample=0.7;, score=-0.408 total time=   5.4s
[CV 7/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=2, min_child_weight=4.818676232053935, n_estimators=500, subsample=0.7;, score=-0.316 total time=   5.3s
[CV 8/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=2, min_

C:\Users\diego\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\diego\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\diego\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\diego\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "C:\Users\diego\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "C:\Users\diego\anaconda3\lib\site-packages\xgboost\sklearn.py", line 1114, in predict
    predts = self.get_booster().inplace_predict(
  File "C:\Users\di

[CV 2/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=2, min_child_weight=4.818676232053935, n_estimators=700, subsample=0.6;, score=nan total time=   7.5s
[CV 3/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=2, min_child_weight=4.818676232053935, n_estimators=700, subsample=0.6;, score=nan total time=   0.0s
[CV 4/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=2, min_child_weight=4.818676232053935, n_estimators=700, subsample=0.6;, score=nan total time=   0.0s
[CV 5/10] END colsample_bytree=0.5, gamma=0.0, learning_rate=0.005, max_depth=2, min_child_weight=4.818676232053935, n_estimators=700, subsample=0.6;, score=nan total time=   0.0s


MemoryError: Unable to allocate 1.60 MiB for an array with shape (209932,) and data type float64

In [14]:
#This chunk is predicting the test csv
validation=pd.read_csv("validate.csv")
count= tfidf_vectorizer.fit_transform(validation['predictor'])
pred1 = grid.predict(count)

ValueError: Feature shape mismatch, expected: 19456, got 12474

In [32]:
pred1.round()

array([2., 2., 2., 2., 2., 2., 2., 3., 2., 2., 2., 3., 2., 2., 2., 2., 2.,
       3., 2., 2., 2., 2., 2., 3., 2., 2., 2., 2., 2., 2., 3., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 3., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 3., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2.

In [34]:
count

<487x12000 sparse matrix of type '<class 'numpy.float64'>'
	with 114622 stored elements in Compressed Sparse Row format>

In [33]:
# Criterion 2 model

y=round(df['rating_commits_to_action'])
X_train, X_test, y_train, y_test= train_test_split(
                                                df['predictor'], y,
                                                test_size=0.33,
                                                random_state=53)
count_vectorizer = CountVectorizer(stop_words='english', max_features=12000)
count_train = count_vectorizer.fit_transform(X_train.values)
count_test = count_vectorizer.transform(X_test.values)


tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7, max_features=12000)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)



nb_classifier = xgb.XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
nb_classifier.fit(count_train, y_train)
pred2 = nb_classifier.predict(count_test)
#metrics.accuracy_score(y_test, pred)


In [34]:
pred2.round()

array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 4., 3., 3., 3., 3., 3., 3.,
       4., 3., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 4., 3., 3., 3., 3.,
       3., 3., 3., 3., 4., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       4., 3., 4., 3., 3., 3., 3., 4., 4., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 4., 3., 4., 3., 3., 3., 4., 4., 3., 3., 3., 3., 3., 3., 3., 4.,
       2., 4., 3., 3., 3., 4., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 4., 3., 3., 3., 3., 3., 3., 3., 4., 3., 3., 3., 3., 3.,
       3., 3., 4., 4., 3., 3., 3., 3., 4., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 4., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 4., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 4., 3., 2., 3., 3., 3., 3.,
       3., 4., 3., 4., 3., 3., 3., 2., 3., 3., 3., 3., 4., 3., 3., 3., 3.,
       3., 3., 3., 3., 3.

In [35]:
validation=pd.read_csv("validate.csv")
count= tfidf_vectorizer.fit_transform(validation['predictor'])
pred2 = nb_classifier.predict(count)
pred2.round()

array([3., 3., 3., 3., 3., 3., 3., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       2., 3., 3., 3., 3., 3., 2., 3., 3., 3., 3., 2., 2., 3., 3., 3., 3.,
       2., 3., 3., 3., 3., 3., 3., 3., 2., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 2., 3., 3., 3., 3., 3., 3., 3., 3., 2., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 2., 3., 3., 3., 2., 3., 3., 3., 2., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 3., 3., 3., 3., 3., 3., 3.,
       2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2.,
       3., 3., 3., 3., 3.

In [22]:
# Criterion 3 model

y=round(df['rating_gathers_information'])
X_train, X_test, y_train, y_test= train_test_split(
                                                df['predictor'], y,
                                                test_size=0.33,
                                                random_state=53)
count_vectorizer = CountVectorizer(stop_words='english')
count_train = count_vectorizer.fit_transform(X_train.values)
count_test = count_vectorizer.transform(X_test.values)

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
nb_classifier = MultinomialNB()

nb_classifier.fit(count_train, y_train)
pred = nb_classifier.predict(count_test)
metrics.accuracy_score(y_test, pred)

0.47368421052631576

In [23]:
# Criterion 4 model

y=round(df['rating_identifies_issues_opportunities'])
X_train, X_test, y_train, y_test= train_test_split(
                                                df['predictor'], y,
                                                test_size=0.33,
                                                random_state=53)
count_vectorizer = CountVectorizer(stop_words='english')
count_train = count_vectorizer.fit_transform(X_train.values)
count_test = count_vectorizer.transform(X_test.values)

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
nb_classifier = MultinomialNB()

nb_classifier.fit(count_train, y_train)
pred = nb_classifier.predict(count_test)
metrics.accuracy_score(y_test, pred)

0.6469298245614035

In [24]:
# Criterion 5 model

y=round(df['rating_interprets_information'])
X_train, X_test, y_train, y_test= train_test_split(
                                                df['predictor'], y,
                                                test_size=0.33,
                                                random_state=53)
count_vectorizer = CountVectorizer(stop_words='english')
count_train = count_vectorizer.fit_transform(X_train.values)
count_test = count_vectorizer.transform(X_test.values)

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
nb_classifier = MultinomialNB()

nb_classifier.fit(count_train, y_train)
pred = nb_classifier.predict(count_test)
metrics.accuracy_score(y_test, pred)

0.6162280701754386

In [25]:
# Criterion 6 model

y=round(df['rating_involves_others'])
X_train, X_test, y_train, y_test= train_test_split(
                                                df['predictor'], y,
                                                test_size=0.33,
                                                random_state=53)
count_vectorizer = CountVectorizer(stop_words='english')
count_train = count_vectorizer.fit_transform(X_train.values)
count_test = count_vectorizer.transform(X_test.values)

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
nb_classifier = MultinomialNB()

nb_classifier.fit(count_train, y_train)
pred = nb_classifier.predict(count_test)
metrics.accuracy_score(y_test, pred)

0.5153508771929824

In [ ]:
# Criterion 7 model

y=round(df['rating_decision_making_final_score'])
X_train, X_test, y_train, y_test= train_test_split(
                                                df['predictor'], y,
                                                test_size=0.33,
                                                random_state=53)
count_vectorizer = CountVectorizer(stop_words='english')
count_train = count_vectorizer.fit_transform(X_train.values)
count_test = count_vectorizer.transform(X_test.values)

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
nb_classifier = MultinomialNB()

nb_classifier.fit(count_train, y_train)
pred = nb_classifier.predict(count_test)
metrics.accuracy_score(y_test, pred)